In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import time
import os


ULTIMAS 52 SEMANAS

In [59]:
def obtener_tabla_filtrada_completa(filtro_menu_id, filtro_opcion_id):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    estadisticas = {
        "Serve": "statso",
        "Return": "statsw",
        "Breaks": "statsl",
        "More": "statst"
    }

    options = Options()
    options.add_argument('--headless')
    dfs_por_estadistica = {nombre: [] for nombre in estadisticas}

    for url in urls:
        driver = webdriver.Chrome(options=options)
        try:
            driver.get(url)

            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, filtro_menu_id)))
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.CLASS_NAME, "stattab")))

            # Aplicar filtro lateral (ej: superficie)
            driver.find_element(By.ID, filtro_menu_id).click()
            time.sleep(1)
            driver.find_element(By.ID, filtro_opcion_id).click()
            time.sleep(2)

            for nombre_estad, clase in estadisticas.items():
                try:
                    # Cambiar tipo de estadística
                    driver.find_element(By.CLASS_NAME, clase).click()
                    time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    tabla = soup.find("table", {"id": "matches"})
                    if not tabla:
                        continue

                    rows = tabla.find_all("tr")
                    data = []
                    for row in rows:
                        cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                        if cols:
                            data.append(cols)

                    if len(data) <= 1:
                        continue

                    headers = data[0]
                    data_rows = data[1:-1]  # Eliminar la última fila

                    df = pd.DataFrame(data_rows, columns=headers)
                    dfs_por_estadistica[nombre_estad].append(df)

                except Exception as e:
                    print(f"Error con estadística {nombre_estad} en {url}: {e}")

        finally:
            driver.quit()

    # Concatenar los DataFrames por estadística y luego unir por columnas
    dfs_final = []
    for nombre_estad, df_list in dfs_por_estadistica.items():
        if df_list:
            df_concat = pd.concat(df_list, ignore_index=True)
            dfs_final.append(df_concat.reset_index(drop=True))

    if dfs_final:
        df_merged = pd.concat(dfs_final, axis=1)
        return df_merged
    else:
        return pd.DataFrame()

In [60]:
def procesar_filtros_y_guardar(
    filtro_menu_id,
    filtro_id_map,
    carpeta_salida,
    nombre_base
):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_filtro, opcion_id in filtro_id_map.items():
        print(f"Procesando filtro: {nombre_filtro}")
        df = obtener_tabla_filtrada_completa(filtro_menu_id, opcion_id)

        if not df.empty:
            archivo = os.path.join(carpeta_salida, f"{nombre_base}_{nombre_filtro.replace(' ', '_')}.csv")
            df.to_csv(archivo, index=False, encoding="utf-8-sig")
            print(f"Guardado: {archivo}")
        else:
            print(f"No se generó archivo para filtro: {nombre_filtro}")


In [ ]:

# Superficies
filtro_id_map_superficies = {
    "Hard": "surface0",
    "Clay": "surface1",
    "Grass": "surface2"
}

# Tipo de torneo
filtro_id_map_torneos = {
    "Grand Slam": "level0",
    "Masters": "level1"
}

# Ronda del torneo
filtro_id_map_rondas = {
    "final": "round0",
    "semis": "round1",
    "cuartos": "round2",
    "r16": "round3",
    "r32": "round4",
    "r64": "round5",
    "r128": "round6"
}

# Número de sets
filtro_id_map_sets = {
    "best of 3": "sets6"
}

# Ranking del oponente
filtro_id_map_vs_rank = {
    "top 10": "rank1",
    "top 50": "rank3"
}

# Mano dominante / revés
filtro_id_map_vs_hand = {
    "left": "hand1",
    "one_hand bh": "hand2"
}


In [ ]:
procesar_filtros_y_guardar(
    filtro_menu_id="surfacehead",
    filtro_id_map=filtro_id_map_superficies,
    carpeta_salida="Estadisticas_ult_52sem",
    nombre_base="stats"
)

procesar_filtros_y_guardar(
    filtro_menu_id="levelhead",
    filtro_id_map=filtro_id_map_torneos,
    carpeta_salida="Estadisticas_ult_52sem",
    nombre_base="stats"
)

procesar_filtros_y_guardar(
    filtro_menu_id="roundhead",
    filtro_id_map=filtro_id_map_rondas,
    carpeta_salida="Estadisticas_ult_52sem",
    nombre_base="stats"
)

procesar_filtros_y_guardar(
    filtro_menu_id="setshead",
    filtro_id_map=filtro_id_map_sets,
    carpeta_salida="Estadisticas_ult_52sems",
    nombre_base="stats"
)

procesar_filtros_y_guardar(
    filtro_menu_id="rankhead",
    filtro_id_map=filtro_id_map_vs_rank,
    carpeta_salida="Estadisticas_ult_52sem",
    nombre_base="stats_vs"
)

procesar_filtros_y_guardar(
    filtro_menu_id="handhead",
    filtro_id_map=filtro_id_map_vs_hand,
    carpeta_salida="Estadisticas_ult_52sem",
    nombre_base="stats_vs"
)


Procesando filtro: Hard
Guardado: ultimas 52 semanas\stats_Hard.csv
Procesando filtro: Clay
Guardado: ultimas 52 semanas\stats_Clay.csv
Procesando filtro: Grass
Guardado: ultimas 52 semanas\stats_Grass.csv
Procesando filtro: Grand Slam
Guardado: ultimas 52 semanas\stats_Grand_Slam.csv
Procesando filtro: Masters
Guardado: ultimas 52 semanas\stats_Masters.csv
Procesando filtro: Final
Guardado: ultimas 52 semanas\stats_Final.csv
Procesando filtro: SF
Guardado: ultimas 52 semanas\stats_SF.csv
Procesando filtro: QF
Guardado: ultimas 52 semanas\stats_QF.csv
Procesando filtro: R16
Guardado: ultimas 52 semanas\stats_R16.csv
Procesando filtro: R32
Guardado: ultimas 52 semanas\stats_R32.csv
Procesando filtro: R64
Guardado: ultimas 52 semanas\stats_R64.csv
Procesando filtro: R128
Guardado: ultimas 52 semanas\stats_R128.csv
Procesando filtro: Best of 3
Guardado: ultimas 52 semanas\stats_Best_of_3.csv
Procesando filtro: Top 10
Guardado: ultimas 52 semanas\stats_vs_Top_10.csv
Procesando filtro: Top 

PARA LAS ESTADISTICAS FILTRADAS DE 2024 Y 2025

In [47]:
def extraer_estadisticas_personalizadas(tipo_filtro: str, filtro_id: str, nombre_filtro: str, filtro_tiempo_id: str, output_path: str):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(options=chrome_options)

    try:
        tablas_totales = []

        for url in urls:
            driver.get(url)
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "spanhead")))

            # Aplicar filtro de tiempo
            driver.find_element(By.ID, "spanhead").click()
            time.sleep(0.5)
            driver.find_element(By.ID, filtro_tiempo_id).click()
            time.sleep(2)

            # Aplicar filtro dinámico (superficie o torneo)
            filtro_categoria_head = {
                "superficie": "surfacehead",
                "torneo": "levelhead",
                "tiempo": "spanhead",  # No se usa como filtro externo aquí, pero se puede ampliar
                "ronda": "roundhead",
                "sets": "setshead",
                "vs_rank": "rankhead",
                "vs_hand": "handhead"
            }

            driver.find_element(By.ID, filtro_categoria_head[tipo_filtro]).click()
            time.sleep(0.5)
            driver.find_element(By.ID, filtro_id).click()
            time.sleep(3)

            # Aplicar los 4 subfiltros de estadísticas (Serve, Return, Breaks, More)
            secciones_stats = ["statso", "statsw", "statsl", "statst"]
            df_combinado = None

            for stat_id in secciones_stats:
                try:
                    driver.find_element(By.CLASS_NAME, stat_id).click()
                    time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    tabla = soup.find("table", {"id": "matches"})

                    if tabla:
                        rows = tabla.find_all("tr")
                        data = []
                        for row in rows:
                            cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                            if cols:
                                data.append(cols)
                        if not data:
                            continue
                        headers = data[0]
                        data_rows = data[1:-1]  # Excluir la última fila

                        df = pd.DataFrame(data_rows, columns=headers)

                        if df_combinado is None:
                            df_combinado = df
                        else:
                            df_combinado = df_combinado.merge(df, on="Rk", suffixes=("", "_dup"), how="left")

                except Exception as e:
                    print(f"Error en sección {stat_id} para filtro {nombre_filtro}: {e}")

            if df_combinado is not None:
                tablas_totales.append(df_combinado)

        df_final = pd.concat(tablas_totales, axis=0, ignore_index=True)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"[✓] Guardado: {output_path}")

    finally:
        driver.quit()


In [53]:
# Tipo de filtro: puede ser 'superficie', 'torneo', 'tiempo'
tipo_filtro = "vs_hand"  # <-- Cambia esto a "superficie" si lo deseas

# Diccionarios de ID y nombres visibles en el menú lateral de la web
filtros = {
    "superficie": {
        "surface0": "Hard",
        "surface1": "Clay",
        "surface2": "Grass"
    },
    "torneo": {
        "level0": "GrandSlams",
        "level1": "Masters",
    },
    "tiempo": {
        "span2025qq": "2025",
        "span2024qq": "2024",
        "span2023qq": "2023"
    },
    "ronda": {
        "round0": "final",
        "round1": "semis",
        "round2": "cuartos",
        "round3": "r16",
        "round4": "r32",
        "round5": "r64",
        "round6": "r128"
    },
    "sets": {
        "sets6": "best_of_3"
    },
    "vs_rank": {
        "rank1": "vs_top_10",
        "rank3": "vs_top_50"
    },
    "vs_hand": {
        "hand1": "vs_left",
        "hand2": "vs_one_hand_bh"
    }
    
}

# Filtro de tiempo que se aplica siempre (en este ejemplo: 2024)
filtro_tiempo_id = "span2025qq"

# Carpeta donde se guardan todos los CSVs
carpeta = "Estadisticas_2025"
os.makedirs(carpeta, exist_ok=True)

for filtro_id, nombre in filtros[tipo_filtro].items():
    ruta_csv = os.path.join(carpeta, f"stats_{nombre}.csv")
    extraer_estadisticas_personalizadas(tipo_filtro, filtro_id, nombre, filtro_tiempo_id, ruta_csv)



[✓] Guardado: Estadisticas_2025\stats_vs_left.csv
[✓] Guardado: Estadisticas_2025\stats_vs_one_hand_bh.csv


PARA LAS ESTADISTICAS GENERALES DE 2024 Y 2025

In [55]:


def extraer_estadisticas_generales(output_path):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=chrome_options)

    try:
        tablas_totales = []

        for url in urls:
            driver.get(url)
            WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.ID, "spanhead")))

            # Aplicar filtro de año 2024
            driver.find_element(By.ID, "spanhead").click()
            time.sleep(0.5)
            # driver.find_element(By.ID, "span2024qq").click()
            driver.find_element(By.ID, "span2025qq").click()
            time.sleep(2)

            # Aplicar los 4 subfiltros de estadísticas
            secciones_stats = ["statso", "statsw", "statsl", "statst"]
            df_combinado = None

            for stat_id in secciones_stats:
                try:
                    driver.find_element(By.CLASS_NAME, stat_id).click()
                    time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    tabla = soup.find("table", {"id": "matches"})

                    if tabla:
                        rows = tabla.find_all("tr")
                        data = []
                        for row in rows:
                            cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                            if cols:
                                data.append(cols)

                        if not data:
                            continue

                        headers = data[0]
                        data_rows = data[1:-1]  # Quita la última fila

                        df = pd.DataFrame(data_rows, columns=headers)

                        if df_combinado is None:
                            df_combinado = df
                        else:
                            df = df.drop(columns=["Rk", "Player"], errors="ignore")
                            df_combinado = pd.concat([df_combinado, df], axis=1)

                except Exception as e:
                    print(f"Error en sección {stat_id} para {url}: {e}")

            if df_combinado is not None:
                tablas_totales.append(df_combinado)

        df_final = pd.concat(tablas_totales, axis=0, ignore_index=True)
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
        print(f"Guardado en: {output_path}")

    finally:
        driver.quit()


In [56]:
# extraer_estadisticas_generales("Estadisticas_2024/estadisticas_generales_2024.csv")
extraer_estadisticas_generales("Estadisticas_2025/estadisticas_generales_2025.csv")



Guardado en: Estadisticas_2025/estadisticas_generales_2025.csv


PARA LAS ESTADISTICAS GENERALES DE ULT 52 SEMANAS

In [63]:


def scrape_principal_stats_sin_filtros(output_path="estadisticas_sin_filtro.csv"):
    urls = [
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi",
        "https://www.tennisabstract.com/cgi-bin/leaders.cgi?players=51-100"
    ]

    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    driver = webdriver.Chrome(options=options)

    tablas_totales = []

    try:
        for url in urls:
            driver.get(url)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "matches")))
            time.sleep(2)

            stat_tabs = ["Serve", "Return", "Breaks", "More"]
            stat_ids = ["statso", "statsw", "statsl", "statst"]

            df_combinado = None

            for idx, stat_id in enumerate(stat_ids):
                try:
                    if idx != 0:
                        driver.find_element(By.CLASS_NAME, stat_id).click()
                        time.sleep(2)

                    soup = BeautifulSoup(driver.page_source, "lxml")
                    table = soup.find("table", {"id": "matches"})

                    if not table:
                        print(f"No se encontró tabla en {stat_tabs[idx]} ({url})")
                        continue

                    rows = table.find_all("tr")
                    data = []
                    for row in rows:
                        cols = [col.get_text(strip=True) for col in row.find_all(["td", "th"])]
                        if cols:
                            data.append(cols)

                    headers = data[0]
                    data_rows = data[1:-1]  # Elimina última fila

                    df = pd.DataFrame(data_rows, columns=headers)

                    if df_combinado is None:
                        df_combinado = df
                    else:
                        df = df.drop(columns=["Rk", "Player"], errors="ignore")
                        df_combinado = pd.concat([df_combinado, df], axis=1)

                except Exception as e:
                    print(f"Error en pestaña {stat_tabs[idx]}: {e}")

            if df_combinado is not None:
                tablas_totales.append(df_combinado)

    finally:
        driver.quit()

    df_final = pd.concat(tablas_totales, axis=0, ignore_index=True)
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    df_final.to_csv(output_path, index=False, encoding="utf-8-sig")
    print(f"[✓] Guardado en: {output_path}")


In [ ]:
scrape_principal_stats_sin_filtros("Estadisticas_ult_52sem/estadisticas_generales_52s.csv")


[✓] Guardado en: Estadisticas_ult_52sem/estadisticas generales.csv
